In [ ]:
'''Statistical tests'''
# when downloading a batch of data, a csv that has been automatically created by the database is also downloaded, containing information about the subjects

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ad_df=pd.read_csv(r'~\AD_data.csv')
cn_df=pd.read_csv(r'~\CN_data.csv');print(ad_df.head,cn_df.head)

In [ ]:
# Drop all rows with only NaN values
ad_df.dropna(how='all', inplace=True)
cn_df.dropna(how='all', inplace=True)

In [ ]:
# Create a dataframe with Subject, Group (AD or CN), Sex and Age
ad_df = ad_df.loc[:, ['Subject', 'Group', 'Sex', 'Age']]
cn_df = cn_df.loc[:, ['Subject', 'Group', 'Sex', 'Age']]

In [ ]:
from scipy.stats import ttest_ind

# Select the 'Age' column from each dataframe
ad_age = ad_df['Age']
cn_age = cn_df['Age']

# Perform the t-test
t, p = ttest_ind(ad_age, cn_age)

# Print the results
print('t-statistic: ', t)
print('p-value: ', p)

In [ ]:
from scipy.stats import chi2_contingency

# Create a contingency table of the Sex variable
contingency_table = pd.crosstab(index=[ad_df['Sex']], columns=[cn_df['Sex']])

# Perform the chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print('Chi-square statistic: ', chi2)
print('p-value: ', p)
print('Degrees of freedom: ', dof)
print('Expected values: ')
print(expected)

In [ ]:
print(np.mean(ad_age),np.std(ad_age),sum(ad_df['Sex']=='F'))
print(np.mean(cn_age),np.std(cn_age),sum(cn_df['Sex']=='F'))

In [ ]:
'''Randomization for creating training, validation & test set'''

In [ ]:
import random
l=["sub-"+str(f"{i+1:02}") for i in range(46)] # creating a list with elements of the form sub-01, sub-02,...,sub-46
random.shuffle(l) # shuffling the list
train=l[:37] # number for slicing is just an example here
val=l[37:41]
test=l[41:]
print("training set: ",train,"\n\nvalidation set: ", val,"\n\ntest set: ", test)

In [ ]:
'''NIFTI FILE HEADER HANDLING'''
# Some Nifti files have wrong TRs
# change header information (specifically pixdim4 which corresponds to TR, can check in linux:  fslhd -x pixdim4 filename.nii.gz)

In [ ]:
import nibabel as nib
import os
path=r'~\data'
for i in ['AD','CN']:
    for j in os.listdir(path+'\\'+i):
        p=path+'\\'+i+'\\'+j
        img = nib.load(p)
        header = img.header
        header['pixdim']=[1., 1., 1., 1., 3., 1., 1., 1.] # changing pixdim4
        #print(header)
        nib.save(img, p) # saving the new nifti file, overwriting the old one

In [ ]:
'''CONVERSION OF 4D TO 2D IMAGES'''

In [ ]:
import numpy as np
import nibabel as nib
import glob
import os 
import cv2
import time
import argparse

In [ ]:
# zt-plane
def imgconverting_z(foldername , targetfolder, prefix, pattern, slicedrop = 10, printfilename=True):
    start = time.time()
    folder = foldername
    os.chdir(folder)
    listfiles = glob.glob('*filtered_func_data.nii.gz') # modification
    listshort = listfiles 
    pref = prefix
    if not os.path.isdir(targetfolder):
        os.mkdir(targetfolder)
    if not os.path.isdir(targetfolder):
        os.mkdir(targetfolder)
    for hh in range(len(listshort)):
        img_test = nib.load(listshort[hh]).get_data()
        print(np.shape(img_test))
        slice3 = np.shape(img_test)[2]
        slice4 = np.shape(img_test)[3]
        if img_test.sum() != 0:
            img_test = (img_test/np.amax(img_test))*255
        for ii in range(slice3-slicedrop):
            for jj in range(slice4):
                img_png = img_test[:,:,ii,jj]
                img_png = np.float64(img_png)
                img_png = img_png.astype(np.uint8)
                if not pattern:
                    helper = ('%s_%s_%03d_%03d_%03d.png' % (
                    pref, listshort[hh].replace('.nii.gz', ''), hh + 1, ii + 1, jj + 1))
                    filename = os.path.join(targetfolder, helper)
                else:
                    helper = ('%s_%s_%03d_%03d_%03d.png' % (pref, pattern, hh + 1, ii + 1, jj + 1))
                    filename = os.path.join(targetfolder, helper)
                img_png = img_png.transpose()
                img = cv2.flip(img_png,0)
                if img.sum() != 0:
                    cv2.imwrite(filename,img_png)
                    if printfilename:
                        print(filename)
                else:
                    if printfilename:
                        print("Empty Slice Skipped => \t", filename)
    end = time.time()
    print("Time :", end-start)

def main(parserargs):
    ff = r'{}'.format(parserargs.inputfolder)
    pref = parserargs.prefix
    pt = parserargs.pattern
    targetfolder = r'{}'.format(parserargs.targetfolder)
    imgconverting(ff, targetfolder, pref, pt, slicedrop=parserargs.drop, printfilename=parserargs.verbose)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("-i", "--inputfolder", required=True, help="Input folder containing nii.gz files")
    parser.add_argument("-o", "--targetfolder", required=True, help="Target folder to store PNG slices")
    parser.add_argument("-p", "--prefix", required=True, help="Prefix for PNG slices")
    parser.add_argument("-t", "--pattern", default=False, help="String pattern as suffix for PNG slices. Default is False : original file name is kept")
    parser.add_argument("-d", "--drop", type=int, default=10, help="Number of slices to drop from the beginning of Nifti. Default is 10 slices")
    parser.add_argument("-v","--verbose",type=bool, default=True, help="Print file names. Default is True")
    args = parser.parse_args()
    main(args)